In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np
import glob
import os
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
import sys
sys.path.append('../scripts/')
from torch_bad_ones import find_bad_ones_torch
from torch.utils.data import Dataset, DataLoader
import PIL
import collections
import math
from torch.nn import Module, Sequential
from torch.optim import Adam
from torchsummary import summary

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
train_files = glob.glob('../../ml-data-training/ship_segmentation_data/train_v2/*.jpg')

In [3]:
print(len(train_files))

192556


In [4]:
train_csv = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')

In [5]:
train_csv.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
updated_train_csv = train_csv.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()

In [7]:
def create_classification_labels(data):
    try:
        if math.isnan(data[0]) == True:
            return 0
    except:
        return 1
    # else:
    #     return 1

In [8]:
updated_train_csv['class_labels'] = updated_train_csv['EncodedPixels'].progress_apply(create_classification_labels)

100%|██████████| 192556/192556 [00:00<00:00, 288988.16it/s]


In [9]:
updated_train_csv

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0
...,...,...,...
192551,fffedbb6b.jpg,[nan],0
192552,ffff2aa57.jpg,[nan],0
192553,ffff6e525.jpg,[nan],0
192554,ffffc50b4.jpg,[nan],0


In [10]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0


In [11]:
updated_train_csv['fixed_paths'] = updated_train_csv['ImageId'].progress_apply(lambda x: f"../../ml-data-training/ship_segmentation_data/train_v2/" + x)

100%|██████████| 192556/192556 [00:00<00:00, 310625.26it/s]


In [12]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels,fixed_paths
0,00003e153.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
1,0001124c7.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1,../../ml-data-training/ship_segmentation_data/...
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1,../../ml-data-training/ship_segmentation_data/...
4,0001b1832.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...


In [13]:
for x in updated_train_csv['fixed_paths'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [14]:
to_check = updated_train_csv['fixed_paths'].values.tolist()

In [15]:
issues = find_bad_ones_torch(to_check)

In [16]:
updated_train_csv = updated_train_csv[updated_train_csv.fixed_paths != issues[0]]

In [17]:
def split_datasets(data, test_size = 0.01):
    train, test = train_test_split(data, test_size = test_size, random_state = 42) 
    train, val = train_test_split(train, test_size = test_size, random_state = 42)
    return train, val, test

In [18]:
train, val, test = split_datasets(updated_train_csv)

In [69]:
class ShipDataset(Dataset):
    def __init__(self, csv_file, return_label = True):
        self.csv_file = csv_file
        self.img_paths = self.csv_file['fixed_paths'].values.tolist()
        self.labels = self.csv_file['class_labels'].values.tolist()
        self.return_label = return_label

    def __getitem__(self, idx):
        img = PIL.Image.open(self.img_paths[idx])
        img = img.resize((16, 16)) 
        img = np.array(img)
        img = img / 255
        # numpy format: H X W X C
        # torch format: C X H X W
        img = img.transpose((2, 0, 1))
        label = self.labels[idx]
        img_tensor = torch.Tensor(img)
        label_tensor = torch.Tensor([label])
        if self.return_label:
            return tuple((img_tensor, label_tensor))
        else:
            return tuple((img_tensor))
        
    def __len__(self):
        return len(self.img_paths)

In [70]:
train_dataset = ShipDataset(train)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
val_dataset = ShipDataset(val, return_label = False)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=32)

In [81]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            torch.nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten()
        )

        self.linear_layers = Sequential(
            torch.nn.Linear(64, 1)
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [82]:
model = Net()
optimizer = Adam(model.parameters(), lr=0.07)
criterion = torch.nn.BCELoss()
if torch.cuda.is_available():
    model = model
    criterion = criterion
# print(model)

In [74]:
for X_batch, y_batch in train_dataloader:
    print(X_batch)
    break

tensor([[[[0.1098, 0.1176, 0.1333,  ..., 0.1294, 0.1176, 0.1176],
          [0.1137, 0.1216, 0.1216,  ..., 0.1255, 0.1176, 0.1216],
          [0.1137, 0.1098, 0.1059,  ..., 0.1176, 0.1216, 0.1137],
          ...,
          [0.1020, 0.1059, 0.0980,  ..., 0.1020, 0.0980, 0.0980],
          [0.1216, 0.1059, 0.0980,  ..., 0.1020, 0.1098, 0.0941],
          [0.1059, 0.0980, 0.1059,  ..., 0.1137, 0.1020, 0.0941]],

         [[0.1608, 0.1647, 0.1804,  ..., 0.1725, 0.1608, 0.1608],
          [0.1608, 0.1686, 0.1686,  ..., 0.1686, 0.1647, 0.1608],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1686, 0.1569],
          ...,
          [0.1569, 0.1608, 0.1529,  ..., 0.1569, 0.1569, 0.1529],
          [0.1725, 0.1608, 0.1529,  ..., 0.1569, 0.1608, 0.1529],
          [0.1647, 0.1529, 0.1569,  ..., 0.1608, 0.1569, 0.1529]],

         [[0.2039, 0.2039, 0.2157,  ..., 0.2235, 0.2078, 0.2039],
          [0.2039, 0.2078, 0.2118,  ..., 0.2196, 0.2118, 0.2078],
          [0.2000, 0.2000, 0.2039,  ..., 0

In [86]:
n_epochs = 200
loss_fn = torch.nn.BCELoss()
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in train_dataloader:
        # X_batch = X_batch.cuda()
        # y_batch = y_batch.cuda()
        y_pred = model(X_batch)
        print(y_pred)
        break
        # loss = loss_fn(y_pred, y_batch)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

tensor([[ 0.6532],
        [ 0.4943],
        [ 2.0170],
        [ 0.5095],
        [ 0.5247],
        [ 0.4479],
        [ 0.2851],
        [ 0.4339],
        [ 0.2405],
        [ 0.1681],
        [ 0.4292],
        [ 0.4744],
        [ 0.3150],
        [ 0.1757],
        [ 0.4100],
        [ 0.9450],
        [ 0.5860],
        [-0.1865],
        [ 0.5570],
        [ 0.3164],
        [ 0.1134],
        [ 0.1996],
        [ 1.4428],
        [ 0.3885],
        [ 0.1763],
        [ 0.4872],
        [ 0.3851],
        [ 0.3373],
        [ 0.5067],
        [ 0.6140],
        [ 0.3885],
        [ 0.3602]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2947],
        [ 0.3076],
        [ 0.7733],
        [ 0.2443],
        [-0.0229],
        [ 0.4992],
        [ 0.3137],
        [ 0.2864],
        [-0.0112],
        [ 0.3665],
        [ 0.3651],
        [-0.0101],
        [ 0.2463],
        [ 0.3258],
        [ 0.8177],
        [ 0.1844],
        [ 0.3690],
        [ 0.3773],
        [ 0.4856],
    

KeyboardInterrupt: 

In [47]:
model.cnn_layers[-1]

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)